<a href="https://colab.research.google.com/github/pdegen/SynEnrich/blob/main/enrichment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, sys, glob
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from datetime import datetime
from google.colab import drive
drive.mount('/content/drive')

def npg_palette():
    palette = ["#E64B35FF", "#4DBBD5FF", "#00A087FF", "#3C5488FF", "#F39B7FFF",
               "#8491B4FF", "#91D1C2FF", "#DC0000FF", "#7E6148FF", "#B09C85FF"]
    return sns.color_palette(palette, len(palette))

npg = npg_palette()
sns.set_style("whitegrid", {'axes.linewidth': 2, 'axes.edgecolor':'black'})

%load_ext rpy2.ipython

# Prepare

In [ ]:
PATH = "/content/drive/MyDrive/rnaseq_chiara"

paramset = "p1" # Kallisto

csvpath = PATH + f"/CSV/{paramset}"
plotpath = PATH + f"/Figures/{paramset}"

contrasts = ["KO_WT","SA_WT","SD_WT","SD_SA","KO_SD","KO_SA"]

gene_dict_path = f"{csvpath}/gene_dict.{paramset}.csv"

fdr_cutoff = 0.05 # cutoff for ClusterProfiler

In [ ]:
test = "qlf"
lfc = 0

for c in contrasts:
  outfile = f"{csvpath}/DEGs/edger.{test}.lfc{lfc}.{c}.{paramset}.csv"
  tab = pd.read_csv(outfile, index_col=0).sort_values(by="logFC")

  #tab["logFC"].to_csv(f"{csvpath}/Enrichment/gsea_lfc.{c}.edger.{test}.{paramset}.csv", header=False)

  tab["signed_logpval"] = -np.sign(tab["logFC"]) * np.log10(tab["PValue"])
  #tab["signed_logpval"].to_csv(f"{csvpath}/Enrichment/gsea_signed_logpval.{c}.edger.{test}.{paramset}.csv", header=False)

  #tab["signed_logpvaladj"] = -np.sign(tab["logFC"]) * np.log10(tab["FDR"])
  #tab["signed_logpvaladj"].to_csv(f"{csvpath}/Enrichment/gsea_signed_logpvaladj.{c}.edger.{test}.{paramset}.csv", header=False)

In [ ]:
f = f"{csvpath}/Enrichment/gsea_{metric}.{c}.edger.qlf.{paramset}.csv"
tab = pd.read_csv(f, index_col=0, header=None)

tab.index = tab.index.map(gene_dict['gene_name'].to_dict())
tab = tab.reset_index().dropna()
tab = tab.set_index(0)

print(len(tab))
tab = tab.loc[~tab.index.duplicated()]
tab = tab.sort_values(by=1,ascending=False)
tab

# STRING


ALthough STRING provides an [API](https://string-db.org/cgi/help?sessionId=bP1AEa0XqxI7&subpage=api), I couldn't find a method to perform functional scoring analysis (only over-representation analysis). Hence, you must use the web interface instead: [Proteins with Values/Ranks - Functional Enrichment Analysis](https://string-db.org/cgi/input?sessionId=blqQmZLHEd5T&input_page_active_form=proteins_with_values)

# ClusterProfiler

Gene list must be in [decreasing order](https://yulab-smu.top/biomedical-knowledge-mining-book/faq.html#genelist)

In [ ]:
%%R
rpath = "/content/drive/MyDrive/single-nuclei/RPackages/"
.libPaths(rpath)

# start_time <- Sys.time()

#install.packages("ggsci", lib = rpath)

# if (!require("BiocManager", quietly = TRUE))
#     install.packages("BiocManager", lib = rpath)

#library(BiocManager, lib.loc = rpath)
#BiocManager:lib = rpath:install("clusterProfiler", lib = rpath)
#BiocManager::install("org.Hs.eg.db", lib=rpath)
#BiocManager::install("biomaRt", lib=rpath)

# end_time <- Sys.time()
# print(end_time - start_time)

In [ ]:
%%R -i gene_dict_path

gene_dict = read.csv(gene_dict_path)
head(gene_dict)
library(org.Hs.eg.db)
keytypes(org.Hs.eg.db)

In [ ]:
%%R

if (!("entrezgene_id" %in% colnames(gene_dict))) {
  print("Appending entrez IDs...")
  library(biomaRt)
  ensembl <- useMart("ensembl", dataset = "hsapiens_gene_ensembl")

  ensg_ids <- gene_dict$gene_id

  gene_info <- getBM(attributes = c('ensembl_gene_id', 'entrezgene_id'),
                    filters = 'ensembl_gene_id',
                    values = ensg_ids,
                    mart = ensembl)

  gene_dict <- merge(gene_dict, gene_info, by.x = "gene_id", by.y = "ensembl_gene_id", all.x = TRUE)
  write.csv(gene_dict,gene_dict_path, row.names = FALSE)
}

In [ ]:
%%R -i csvpath,paramset,contrasts,fdr_cutoff

library(clusterProfiler, lib=rpath)

organism <- "org.Hs.eg.db"
organism_kegg <- "hsa"

#metric <- "lfc"
metric <- "signed_logpval"
#metric <- "signed_logpvaladj"

overwrite = FALSE

for (c in contrasts) {

  outfile <- paste0(csvpath,"/Enrichment/clusterprofiler/cluster.gseGO.",metric,".",c,".",paramset,".csv")
  outfile_kegg <- paste0(csvpath,"/Enrichment/clusterprofiler/cluster.gseKEGG.",metric,".",c,".",paramset,".csv")

  if (file.exists(outfile) && file.exists(outfile_kegg) && !overwrite) {
    print("Existing files not overwritte, skipping")
    next
  }

  start_time <- Sys.time()
  p <- paste0(csvpath,"/Enrichment/gsea_",metric,".",c,".edger.qlf.",paramset,".csv")
  print(p)

  d = read.csv(p, header=FALSE, col.names=c("gene_id", "metric"))
  print(paste("ENSMBL genes:", nrow(d)))

  # Convert to ENTREZ ID
  # We will lose some genes here because not all IDs will be converted
  ids<-bitr(d$gene_id, fromType = "ENSEMBL", toType = "ENTREZID", OrgDb=organism)
  dedup_ids = ids[!duplicated(ids[c("ENSEMBL")]),]
  df2 = d[d$gene_id %in% dedup_ids$ENSEMBL,]
  df2$Y = dedup_ids$ENTREZID

  #d <- d[order(d$metric, decreasing = TRUE), ]
  #print(head(d))

  geneList <- df2$metric
  names(geneList) <- df2$Y
  geneList<-na.omit(geneList)
  geneList = sort(geneList, decreasing = TRUE)

  #print(head(geneList))
  print(paste("ENTREZ genes:", length(geneList)))

  if (!file.exists(outfile) || overwrite) {

    ego3 <- gseGO(geneList     = geneList,
                  OrgDb        = organism,
                  ont          = "ALL", ## CC MF BP
                  minGSSize    = 100,
                  maxGSSize    = 500,
                  pvalueCutoff = fdr_cutoff,
                  nPerm = 1000,
                  verbose      = FALSE)
    write.csv(ego3,outfile)
  }

  if (!file.exists(outfile_kegg) || overwrite) {

    kegg <- gseKEGG(geneList     = geneList,
                  organism        = organism_kegg,
                  minGSSize    = 100,
                  maxGSSize    = 500,
                  pvalueCutoff = fdr_cutoff,
                  nPerm = 1000,
                  verbose      = FALSE)
    write.csv(kegg,outfile_kegg)
  }

  end_time <- Sys.time()
  print(end_time - start_time)
}

#GSEApy

Note: GSEApy ranks in ascending=False order (as of commit 275972e)

https://github.com/zqfang/GSEApy/blob/master/gseapy/gsea.py#L356

https://github.com/zqfang/GSEApy/blob/master/gseapy/gsea.py#L398

In [ ]:
try:
  import gseapy
except ModuleNotFoundError:
  !pip install gseapy
  import gseapy

In [ ]:
gene_dict = pd.read_csv(gene_dict_path, index_col=0)
print(len(gene_dict))
gene_dict.head()

In [ ]:
ontologies = ["GO_Biological_Process_2023","GO_Cellular_Component_2023",
              "GO_Molecular_Function_2023","KEGG_2021_Human"]

def run_gseapy_multi(contrasts, metric, overwrite=False, **kwargs):

  if isinstance(contrasts, str):
    contrasts = [contrasts]

  for c in contrasts:
    print(c)

    outfile = f"{csvpath}/Enrichment/gseapy.{metric}.{c}.{paramset}.csv"

    if os.path.isfile(outfile) and not overwrite:
      print(f"Existing file not overwritten: {outfile}")
      continue
    else:
      print(f"Running GSEApy")

    start = time.time()

    f = f"{csvpath}/Enrichment/gsea_{metric}.{c}.edger.qlf.{paramset}.csv"
    tab = pd.read_csv(f, index_col=0, header=None)

    tab.index = tab.index.map(gene_dict['gene_name'].to_dict())
    tab = tab.reset_index().dropna()
    tab = tab.set_index(0)

    print(len(tab))
    tab = tab.loc[~tab.index.duplicated()]
    print(len(tab))

    res_list = []
    for ont in ontologies:
      res = run_gseapy(tab, ont, **kwargs)
      res_list.append(res.res2d)

    res_merged = pd.concat(res_list)
    res_merged = res_merged.reset_index(drop=True)

    res_merged.to_csv(outfile)

    end = time.time()
    print("Time difference:", end - start)

def run_gseapy(tab, ontology, outdir=None, **kwargs):
    res = gseapy.prerank(rnk=tab, gene_sets=ontology, outdir=None, **kwargs)
    res.res2d["Ontology"] = ontology
    return res

In [ ]:
#metric = "signed_logpval"
run_gseapy_multi(contrasts, metric, overwrite=False)

In [ ]:
outfile = f"{csvpath}/Enrichment/gseapy.{metric}.KO_WT.{paramset}.csv"
tab=pd.read_csv(outfile,index_col=0)
g=tab[tab["Ontology"].str.startswith("GO_")]
g[g["FDR q-val"]<0.05]

# Method comparison

In [ ]:
try:
  import bioservices
except ModuleNotFoundError:
  !pip install bioservices
  import bioservices

In [ ]:
from bioservices import KEGG

pathway_terms = ["Axon guidance", "Alanine, aspartate and glutamate metabolism", "Basal transcription factors"]

if "kegg_ids_df" not in globals():
  try:
    kegg_ids_df = pd.read_csv(f"{csvpath}/kegg_ids_df.csv", index_col=0)
  except FileNotFoundError:
    print("Creating kegg_ids_df.csv")
    kegg_ids_df = pd.DataFrame(columns=["Term","ID"])

# Function to get KEGG pathway ID from term
# Also returns boolean to indicate if kegg_ids_df df has been updated
def get_kegg_id_hsa(term):

  assert "kegg_ids_df" in globals(), "kegg_ids_df not defined"

  if term in kegg_ids_df["Term"].values:
    return kegg_ids_df[kegg_ids_df["Term"]==term]["ID"].values[0], False
  else:
    print(f"Checking bioservices for term: {term}")
  k = KEGG()
  try:
      term = term.replace(",","") # comma results in 400 bad request
      term = term.replace("/","")

      search_results = k.find("pathway", term)
      # Assuming the first result is the best match
      first_result = search_results.split('\n')[0]
      kegg_id = first_result.split('\t')[0]
      kegg_id = kegg_id.replace("path:map","hsa")
      kegg_ids_df.loc[len(kegg_ids_df)] = [term, kegg_id]
      return kegg_id, True
  except Exception as e:
      print(f"Error retrieving KEGG ID for term '{term}': {e}")
      return None

# Get KEGG IDs for each term
kegg_ids = {term: get_kegg_id_hsa(term)[0] for term in pathway_terms}
print(kegg_ids)

#get_kegg_id_hsa(pathway_terms[1])

In [ ]:
# f = f"{csvpath}/Enrichment/gseapy/gseapy.{metric}.{c}.{paramset}.csv"
# tab_gs = pd.read_csv(f, index_col=0)

# tab_gs = tab_gs[tab_gs["FDR q-val"]<fdr]
# tab_gs = tab_gs[(tab_gs["Ontology"].str.startswith("GO_")) | (tab_gs["Ontology"].str.startswith("KEGG_"))]
# go_ix = tab_gs[tab_gs["Ontology"].str.startswith("GO_")].index
# tab_gs.loc[go_ix, "ID"] = "GO:" + tab_gs["Term"].str.split("\(GO:").str[1].str[:-1]

# tab_gs

In [ ]:
from matplotlib_venn import venn3, venn3_circles

fdr = 0.01 # stringency used for method comparison
metric = "signed_logpval"

results_dict = {c: None for c in contrasts}

for c in contrasts:

  ############ STRING ############
  f = f"{csvpath}/Enrichment/string/{c}.{metric}.enrichment.all.{paramset}.tsv"
  tab = pd.read_csv(f, sep="\t")
  tab = tab[(tab["#category"].str.startswith("GO")) | (tab["#category"].str.startswith("KEGG"))]
  tab = tab[tab["false discovery rate"]<fdr]
  tab = tab.rename({"term ID":"ID"}, axis=1)

  ############ CLUSTERPROFILER ############
  f = f"{csvpath}/Enrichment/clusterprofiler/cluster.gseGO.{metric}.{c}.{paramset}.csv"
  tab_cl = pd.read_csv(f, index_col=0)

  f = f"{csvpath}/Enrichment/clusterprofiler/cluster.gseKEGG.{metric}.{c}.{paramset}.csv"
  tab_cl_kegg = pd.read_csv(f, index_col=0)

  tab_cl = pd.concat([tab_cl,tab_cl_kegg])
  tab_cl = tab_cl[tab_cl["qvalue"]<fdr]

  ############ GSEAPY ############
  f = f"{csvpath}/Enrichment/gseapy/gseapy.{metric}.{c}.{paramset}.csv"
  tab_gs = pd.read_csv(f, index_col=0)

  is_dirty = False
  if "ID" not in tab_gs:
    print("Retrieving KEGG IDs for GSEApy...")
    tab_gs["ID"] = np.NaN
    ix = tab_gs[tab_gs["Ontology"].str.startswith("KEGG_")].index
    for k, i in enumerate(ix):
      term = tab_gs.loc[i, "Term"]
      tab_gs.loc[i,"ID"], new_kegg_id_found = get_kegg_id_hsa(term)
      if new_kegg_id_found:
        is_dirty = True
      if k % 100 == 0: print(k,i,term)
    tab_gs.to_csv(f)

  tab_gs = tab_gs[tab_gs["FDR q-val"]<fdr]
  tab_gs = tab_gs[(tab_gs["Ontology"].str.startswith("GO_")) | (tab_gs["Ontology"].str.startswith("KEGG_"))]
  go_ix = tab_gs[tab_gs["Ontology"].str.startswith("GO_")].index
  tab_gs.loc[go_ix, "ID"] = "GO:" + tab_gs["Term"].str.split("\(GO:").str[1].str[:-1]

  if is_dirty:
    print("Updating kegg_ids_df.csv")
    kegg_ids_df.to_csv(f"{csvpath}/kegg_ids_df.csv")

  # JACCARD
  # inter = set(tab["ID"]).intersection(set(tab_cl["ID"]))
  # union = set(tab["ID"]).union(set(tab_cl["ID"]))

  # print(c)
  # print("STRING:", len(tab), "| ClusterProfiler:", len(tab_cl))
  # print("Intersection", len(inter))
  # print("Union", len(union))
  # print(f"Overlap: {len(inter)/min(len(tab),len(tab_cl)):.2f}")
  # print(f"Jaccard: {len(inter)/len(union):.2f}\n")

  STRING = set(tab["ID"])
  CLUSTER = set(tab_cl["ID"])
  GSEAPY = set(tab_gs["ID"])

  results_dict[c] = {"STRING":tab,"ClusterProfiler":tab_cl,"GSEApy":tab_gs}

  ############ PLOTTING

  fig, ax = plt.subplots(1,1, figsize=(4,4))

  venn3(subsets=[STRING, CLUSTER, GSEAPY], set_labels = ('STRING', 'ClusterProfiler', 'GSEApy'),ax=ax)

  ax.set_title(f"{c}")
  # param_info = f"{c}\nFDR < {fdr:.0%}\nGO (BP,MF,CC)\n{metric}"
  # props = dict(boxstyle='round', facecolor='lightgrey', alpha=0.5)
  # ax.text(1.1, 0.5, param_info, transform=ax.transAxes, fontsize=12,
  #       verticalalignment='center', bbox=props)
  # plt.subplots_adjust(right=0.8)

  fig.tight_layout()
  figpath = f"{plotpath}/venn3.enrichment.{c}.{metric}.{paramset}.pdf"
  fig.savefig(figpath,bbox_inches='tight')
  print(figpath)

## Top terms

In [ ]:
def get_intersection_depths(df_list):

  # Create a defaultdict to count occurrences
  element_counts = defaultdict(int)

  # Count occurrences of elements across subsets
  for subset in subsets:
      for element in subset:
          element_counts[element] += 1

  return element_counts

#for c in contrasts:
c = "KO_WT"

def combine_dfs(results_dict):

  for method in ["STRING","ClusterProfiler","GSEApy"]:

    tab = results_dict[c][method]
    tab["Method"] = method

    match method:
      case "STRING":
        tab.rename({"false discovery rate": "p.adj",
                    "term description":"Term"}, axis=1, inplace=True)
        tab["Direction"] = tab['direction'].apply(lambda x: 'Up' if x == "top" else ('Both' if x == "both ends" else 'Down'))
      case "ClusterProfiler":
        tab.rename({"qvalue": "p.adj",
                    "Description":"Term"}, axis=1, inplace=True)
        tab["Direction"] = tab['NES'].apply(lambda x: 'Up' if x > 0 else ('Both' if x == 0 else 'Down'))
      case "GSEApy":
        tab.rename({"FDR q-val": "p.adj"}, axis=1, inplace=True)
        tab["Direction"] = tab['NES'].apply(lambda x: 'Up' if x > 0 else ('Both' if x == 0 else 'Down'))

  combined= pd.concat([results_dict[c]["STRING"][["ID","p.adj","Method","Direction","Term"]],
                       results_dict[c]["ClusterProfiler"][["ID","p.adj","Method","Direction","Term"]],
                       results_dict[c]["GSEApy"][["ID","p.adj","Method","Direction","Term"]]
                      ])

  combined["ID_Direction"] = combined["ID"] + " " + combined["Direction"]
  combined["Counts"] = combined.groupby(['ID_Direction'])['ID_Direction'].transform('count')
  combined.sort_values(by=["Counts","p.adj"], ascending=[False,True], inplace=True)
  combined = combined[["ID","Counts","p.adj","Direction","Method","Term","ID_Direction"]]
  return combined.reset_index(drop=True)

combined  = combine_dfs(results_dict)

In [ ]:
combined.to_csv(f"{csvpath}/Enrichment/combined.csv")
combined

In [ ]:
combined[combined["Counts"]>2].sort_values(by="ID")

In [ ]:
inter = combined["ID"].value_counts()
inter = inter[inter > 2]
combined[combined["ID"].isin(inter.index.values)]

# Metric comparison

In [ ]:
from matplotlib_venn import venn2

metric1 = "lfc"
metric2 = "signed_logpval"

for c in contrasts:
  f = f"{csvpath}/Enrichment/string/{c}.{metric1}.enrichment.all.{paramset}.tsv"
  tab = pd.read_csv(f, sep="\t")
  tab = tab[(tab["#category"].str.startswith("GO")) | (tab["#category"].str.startswith("KEGG"))]

  f = f"{csvpath}/Enrichment/string//{c}.{metric2}.enrichment.all.{paramset}.tsv"
  tab2 = pd.read_csv(f, sep="\t")
  tab2 = tab2[(tab2["#category"].str.startswith("GO")) | (tab2["#category"].str.startswith("KEGG"))]

  # JACCARD
  inter = set(tab["term ID"]).intersection(set(tab2["term ID"]))
  union = set(tab["term ID"]).union(set(tab2["term ID"]))

  print(c)
  print(f"{metric1}:", len(tab), f"| {metric2}:", len(tab2))
  print("Intersection", len(inter))
  print("Union", len(union))
  print(f"Overlap: {len(inter)/min(len(tab),len(tab2)):.2f}")
  print(f"Jaccard: {len(inter)/len(union):.2f}\n")


  STRING1 = set(tab["term ID"])
  STRING2 = set(tab2["term ID"])

  fig, ax = plt.subplots(1,1, figsize=(4,4))

  venn2(subsets=[STRING1, STRING2], set_labels = ("logFC","signed_logpval"),ax=ax)

  ax.set_title(f"{c}")

  # param_info = f"{c}\nFDR < {fdr:.0%}\nGO (BP,MF,CC)"
  # props = dict(boxstyle='round', facecolor='lightgrey', alpha=0.5)
  # ax.text(1.1, 0.5, param_info, transform=ax.transAxes, fontsize=12,
  #       verticalalignment='center', bbox=props)
  # plt.subplots_adjust(right=0.8)

  fig.tight_layout()
  figpath = f"{plotpath}/venn2.enrichment.{c}.STRING.{paramset}.pdf"
  fig.savefig(figpath,bbox_inches='tight')
  print(figpath)